# Baseline

In [48]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, StratifiedGroupKFold, KFold

## Загрузка данных

In [49]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

In [50]:
train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,199998,month_2,-0.156775,-0.204960,-0.125987,-0.156311,channel_code_9,city_14,city_type_0,None,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{}
599996,199998,month_3,-0.156776,-0.204960,-0.125995,-0.156312,channel_code_9,city_14,city_type_0,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{}
599997,199999,month_1,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_1876,city_type_0,index_city_code_195,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{}
599998,199999,month_2,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_1876,city_type_0,index_city_code_195,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{}


In [51]:
train_df.head(30)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other}
5,1,month_3,-0.090605,-0.114275,-0.114119,-0.089937,channel_code_2,city_14,city_type_0,None,...,0.948027,0.488221,0.043221,0.560788,0.707687,-0.167905,0.259011,0.605309,{other},{other}
6,2,month_1,-0.154685,-0.186795,-0.122805,-0.154215,channel_code_12,city_613,city_type_306,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.178674,0.252657,0.440474,{α},{α}
7,2,month_2,-0.152784,-0.193686,-0.122805,-0.152308,channel_code_12,city_613,city_type_306,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.177854,0.252657,0.440474,{α},{α}
8,2,month_3,-0.148737,-0.187003,-0.112416,-0.148249,channel_code_12,city_613,city_type_306,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.176302,0.252368,0.429485,{α},{α}
9,3,month_1,-0.156643,-0.204861,-0.125660,-0.156179,channel_code_14,city_21,city_type_0,index_city_code_46,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{α}


In [52]:

train_df['month_number'] = train_df.groupby('id').cumcount() + 1


train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster,month_number
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other},1
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other},2
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other},3
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other},1
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other},2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,199998,month_2,-0.156775,-0.204960,-0.125987,-0.156311,channel_code_9,city_14,city_type_0,None,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},2
599996,199998,month_3,-0.156776,-0.204960,-0.125995,-0.156312,channel_code_9,city_14,city_type_0,None,...,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{},3
599997,199999,month_1,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_1876,city_type_0,index_city_code_195,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},1
599998,199999,month_2,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_1876,city_type_0,index_city_code_195,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},2


In [53]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 94 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       600000 non-null  int64  
 1   date                     600000 non-null  object 
 2   balance_amt_avg          534423 non-null  float64
 3   balance_amt_max          534423 non-null  float64
 4   balance_amt_min          534423 non-null  float64
 5   balance_amt_day_avg      534423 non-null  float64
 6   channel_code             557150 non-null  object 
 7   city                     521656 non-null  object 
 8   city_type                521656 non-null  object 
 9   index_city_code          274776 non-null  object 
 10  ogrn_days_end_month      568527 non-null  float64
 11  ogrn_days_end_quarter    568527 non-null  float64
 12  ogrn_month               568527 non-null  object 
 13  ogrn_year                568527 non-null  object 
 14  ft_r

In [54]:
pivot_columns = [col for col in train_df.columns if col not in ['id', 'date', 'month_number']]

df_pivoted = train_df.pivot_table(index='id', columns='month_number', values=pivot_columns, aggfunc='first')

# Теперь, когда мы сформировали сводную таблицу, преобразуем MultiIndex столбцы в единый уровень
df_pivoted.columns = [f'{col[0]}_{col[1]}month' for col in df_pivoted.columns]

# Переименовываем индекс обратно в столбец 'id', если он стал индексом
df_pivoted.reset_index(inplace=True)

df_pivoted.head()



,id,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,...,sum_deb_h_oper_3m_3month,sum_of_paym_1y_1month,sum_of_paym_1y_2month,sum_of_paym_1y_3month,sum_of_paym_2m_1month,sum_of_paym_2m_2month,sum_of_paym_2m_3month,sum_of_paym_6m_1month,sum_of_paym_6m_2month,sum_of_paym_6m_3month
0,0,0.744845,1.049605,0.692653,0.748101,1.053805,0.695747,0.705492,0.831916,0.740253,...,0.877050,0.511490,0.486425,0.480547,0.942275,0.645704,0.403604,0.536013,0.536378,0.613167
1,1,-0.081586,-0.094962,-0.090605,-0.080890,-0.094307,-0.089937,-0.091860,-0.100504,-0.114275,...,0.043221,0.052041,0.033554,0.039472,0.014051,-0.057593,-0.092059,0.043800,0.035027,0.025233
2,2,-0.154685,-0.152784,-0.148737,-0.154215,-0.152308,-0.148249,-0.186795,-0.193686,-0.187003,...,-0.165588,-0.291924,-0.290712,-0.288318,-0.255837,-0.267913,-0.255946,-0.287121,-0.284955,-0.280676
3,3,-0.156643,-0.156666,-0.156522,-0.156179,-0.156202,-0.156058,-0.204861,-0.204790,-0.204718,...,-0.165588,-0.242793,-0.262878,-0.273303,-0.273969,-0.273969,-0.273969,-0.268832,-0.294398,-0.294447
4,4,-0.138847,-0.129233,-0.141798,-0.138328,-0.128684,-0.141289,-0.182486,-0.162965,-0.170262,...,-0.078297,-0.124641,-0.121939,-0.128903,-0.103807,-0.134192,-0.166740,-0.130025,-0.134049,-0.142831


In [55]:
df_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 274 entries, id to sum_of_paym_6m_3month
dtypes: float64(243), int64(1), object(30)
memory usage: 418.1+ MB


In [56]:
cat_cols = [
    "channel_code_2month", "city_2month", "city_type_2month",
    "okved_2month", "segment_2month", "start_cluster_2month",
    "index_city_code_2month", "ogrn_month_2month", "ogrn_year_2month",
    "channel_code_1month", "city_1month", "city_type_1month",
    "okved_1month", "segment_1month", "start_cluster_1month",
    "index_city_code_1month", "ogrn_month_1month", "ogrn_year_1month",
    "channel_code_3month", "city_3month", "city_type_3month",
    "okved_3month", "segment_3month", "start_cluster_3month",
    "index_city_code_3month", "ogrn_month_3month", "ogrn_year_3month",
]

In [77]:
float_cols = df_pivoted.select_dtypes(include=['float64', 'float32'])

std_devs = float_cols.std()
srted = std_devs.sort_values()

srted.head(20)


cnt_cred_f_oper_3m_1month    0.000886
cnt_cred_e_oper_3m_1month    0.001131
cnt_cred_f_oper_1m_1month    0.001246
cnt_deb_f_oper_3m_1month     0.001289
cnt_cred_e_oper_1m_1month    0.001361
cnt_cred_f_oper_1m_2month    0.001488
cnt_deb_f_oper_1m_1month     0.001514
cnt_cred_f_oper_3m_2month    0.001546
cnt_cred_e_oper_1m_2month    0.001563
cnt_b_oper_3m_3month         0.001655
cnt_cred_e_oper_3m_2month    0.001695
cnt_deb_f_oper_1m_2month     0.001834
cnt_deb_f_oper_3m_2month     0.001894
cnt_deb_e_oper_1m_1month     0.002139
cnt_deb_e_oper_3m_1month     0.002175
cnt_b_oper_3m_2month         0.002196
cnt_deb_e_oper_1m_2month     0.002359
cnt_cred_f_oper_3m_3month    0.002564
cnt_deb_f_oper_3m_3month     0.002612
cnt_deb_e_oper_3m_2month     0.002639
dtype: float64

In [81]:
float_cols = df_pivoted.select_dtypes(include=['float64', 'float32'])

means = float_cols.mean()

std_devs = float_cols.std()

cv_percent = (std_devs / abs(means)) * 100

sorted_cv_percent = cv_percent.sort_values()

print(sorted_cv_percent.iloc[0:15])


cnt_cred_e_oper_3m_1month    0.130930
cnt_cred_e_oper_1m_1month    0.155773
cnt_cred_e_oper_1m_2month    0.178981
cnt_cred_e_oper_3m_2month    0.196164
cnt_deb_f_oper_1m_1month     0.196272
cnt_deb_f_oper_1m_2month     0.237819
cnt_deb_e_oper_1m_1month     0.239462
cnt_deb_e_oper_1m_2month     0.264140
cnt_cred_e_oper_3m_3month    0.307015
cnt_cred_e_oper_1m_3month    0.310253
cnt_b_oper_3m_3month         0.319841
cnt_deb_e_oper_1m_3month     0.345459
cnt_deb_f_oper_1m_3month     0.348912
cnt_cred_f_oper_3m_1month    0.356484
cnt_c_oper_3m_2month         0.358865
dtype: float64


Обозначение категориальных признаков

In [57]:
# ["cnt_days_cred_f_oper_3m_1month", "sum_cred_f_oper_1m_1month", "cnt_days_cred_f_oper_1m_1month", "cnt_cred_f_oper_1m_1month",
#  "cnt_cred_d_oper_1m_1month", "cnt_cred_d_oper_3m_1month", "cnt_cred_f_oper_1m_1month", "cnt_cred_g_oper_1m_1month",
#  "cnt_days_cred_f_oper_3m_2month", "sum_cred_f_oper_1m_2month", "cnt_days_cred_f_oper_1m_2month", "cnt_cred_f_oper_1m_2month",
#  "cnt_cred_d_oper_1m_2month", "cnt_cred_d_oper_3m_2month", "cnt_cred_f_oper_1m_2month", "cnt_cred_g_oper_1m_2month",
#  "cnt_days_cred_f_oper_3m_3month", "sum_cred_f_oper_1m_3month", "cnt_days_cred_f_oper_1m_3month", "cnt_cred_f_oper_1m_3month",
#  "cnt_cred_d_oper_1m_3month", "cnt_cred_d_oper_3m_3month", "cnt_cred_f_oper_1m_3month", "cnt_cred_g_oper_1m_3month",
# "sum_cred_d_oper_1m_3month", "cnt_days_deb_f_oper_1m_3month", "sum_cred_g_oper_1m_3month", "cnt_days_cred_g_oper_1m_3month", "sum_a_oper_1m_3month", "sum_b_oper_1m_3month", "cnt_b_oper_1m_3month",
# "sum_cred_d_oper_1m_2month", "cnt_days_deb_f_oper_1m_2month", "sum_cred_g_oper_1m_2month", "cnt_days_cred_g_oper_1m_2month", "sum_a_oper_1m_2month", "sum_b_oper_1m_2month", "cnt_b_oper_1m_2month",
# "sum_cred_d_oper_1m_1month", "cnt_days_deb_f_oper_1m_1month", "sum_cred_g_oper_1m_1month", "cnt_days_cred_g_oper_1m_1month", "sum_a_oper_1m_1month", "sum_b_oper_1m_1month", "cnt_b_oper_1m_1month",

# "cnt_days_cred_h_oper_1m_1month",  "cnt_cred_h_oper_1m_1month","cnt_days_cred_h_oper_1m_2month",  "cnt_cred_h_oper_1m_2month","cnt_days_cred_h_oper_1m_3month",  "cnt_cred_h_oper_1m_3month",

Создаем выборки для валидации и обучения

In [58]:
X = df_pivoted.drop([ "id","end_cluster_3month", "end_cluster_2month", "end_cluster_1month"
], axis=1)
# X = df_pivoted.drop([
#     "cnt_days_cred_h_oper_1m_1month",  "cnt_cred_h_oper_1m_1month","cnt_days_cred_h_oper_1m_2month",  "cnt_cred_h_oper_1m_2month","cnt_days_cred_h_oper_1m_3month",  "cnt_cred_h_oper_1m_3month",
#  "cnt_cred_d_oper_1m_3month", "cnt_cred_d_oper_3m_3month", "cnt_cred_f_oper_1m_3month", "cnt_cred_g_oper_1m_3month",
# "sum_cred_d_oper_1m_3month", "cnt_days_deb_f_oper_1m_3month", "sum_cred_g_oper_1m_3month", "cnt_days_cred_g_oper_1m_3month", "sum_a_oper_1m_3month", "sum_b_oper_1m_3month", "cnt_b_oper_1m_3month",
# "sum_cred_d_oper_1m_2month", "cnt_days_deb_f_oper_1m_2month", "sum_cred_g_oper_1m_2month", "cnt_days_cred_g_oper_1m_2month", "sum_a_oper_1m_2month", "sum_b_oper_1m_2month", "cnt_b_oper_1m_2month",
# "sum_cred_d_oper_1m_1month", "cnt_days_deb_f_oper_1m_1month", "sum_cred_g_oper_1m_1month", "cnt_days_cred_g_oper_1m_1month", "sum_a_oper_1m_1month", "sum_b_oper_1m_1month", "cnt_b_oper_1m_1month","cnt_days_cred_f_oper_3m_1month", "sum_cred_f_oper_1m_1month", "cnt_days_cred_f_oper_1m_1month", "cnt_cred_f_oper_1m_1month",
#  "cnt_cred_d_oper_1m_1month", "cnt_cred_d_oper_3m_1month", "cnt_cred_f_oper_1m_1month", "cnt_cred_g_oper_1m_1month",
#  "cnt_days_cred_f_oper_3m_2month", "sum_cred_f_oper_1m_2month", "cnt_days_cred_f_oper_1m_2month", "cnt_cred_f_oper_1m_2month",
#  "cnt_cred_d_oper_1m_2month", "cnt_cred_d_oper_3m_2month", "cnt_cred_f_oper_1m_2month", "cnt_cred_g_oper_1m_2month",
#  "cnt_days_cred_f_oper_3m_3month", "sum_cred_f_oper_1m_3month", "cnt_days_cred_f_oper_1m_3month", "cnt_cred_f_oper_1m_3month",
#  "cnt_cred_d_oper_1m_3month", "cnt_cred_d_oper_3m_3month", "cnt_cred_f_oper_1m_3month", "cnt_cred_g_oper_1m_3month", "id","end_cluster_3month", "end_cluster_2month", "end_cluster_1month"
# ], axis=1)
y = df_pivoted["end_cluster_3month"]

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучение модели

В качестве базовой модели возьмем LGBM обучим на всех признаках

In [59]:
params_cat = {
    "iterations":1000,
    "depth":12,
    "learning_rate":0.25,
    "eval_metric":'AUC',
    "random_seed":1,
    "bootstrap_type":'Bayesian',
    "bagging_temperature":1,
    "od_type":'Iter',
    "od_wait":50,
    "task_type":'GPU'
}

In [60]:
from sklearn.model_selection import StratifiedKFold, LeavePOut, RepeatedStratifiedKFold
from catboost import  CatBoostClassifier


cat_model = CatBoostClassifier(**params_cat)


# cv = KFold(n_splits=7)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=12)

# cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
auc_scores = []  # Список для хранения показателей AUC для каждого разбиения


In [61]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [62]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [63]:
# weights_dict

In [64]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].astype('category')
    #test_df[col] = test_df[col].astype('category')



In [65]:
y = y.astype('category')
# y = y.cat.add_categories('Missing').fillna('Missing')

In [66]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].cat.add_categories('Missing').fillna('Missing')
cat_features

['channel_code_1month',
 'channel_code_2month',
 'channel_code_3month',
 'city_1month',
 'city_2month',
 'city_3month',
 'city_type_1month',
 'city_type_2month',
 'city_type_3month',
 'index_city_code_1month',
 'index_city_code_2month',
 'index_city_code_3month',
 'ogrn_month_1month',
 'ogrn_month_2month',
 'ogrn_month_3month',
 'ogrn_year_1month',
 'ogrn_year_2month',
 'ogrn_year_3month',
 'okved_1month',
 'okved_2month',
 'okved_3month',
 'segment_1month',
 'segment_2month',
 'segment_3month',
 'start_cluster_1month',
 'start_cluster_2month',
 'start_cluster_3month']

In [67]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
cat_features

['channel_code_1month',
 'channel_code_2month',
 'channel_code_3month',
 'city_1month',
 'city_2month',
 'city_3month',
 'city_type_1month',
 'city_type_2month',
 'city_type_3month',
 'index_city_code_1month',
 'index_city_code_2month',
 'index_city_code_3month',
 'ogrn_month_1month',
 'ogrn_month_2month',
 'ogrn_month_3month',
 'ogrn_year_1month',
 'ogrn_year_2month',
 'ogrn_year_3month',
 'okved_1month',
 'okved_2month',
 'okved_3month',
 'segment_1month',
 'segment_2month',
 'segment_3month',
 'start_cluster_1month',
 'start_cluster_2month',
 'start_cluster_3month']

In [68]:
X

,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,balance_amt_min_1month,...,sum_deb_h_oper_3m_3month,sum_of_paym_1y_1month,sum_of_paym_1y_2month,sum_of_paym_1y_3month,sum_of_paym_2m_1month,sum_of_paym_2m_2month,sum_of_paym_2m_3month,sum_of_paym_6m_1month,sum_of_paym_6m_2month,sum_of_paym_6m_3month
0,0.744845,1.049605,0.692653,0.748101,1.053805,0.695747,0.705492,0.831916,0.740253,1.287207,...,0.877050,0.511490,0.486425,0.480547,0.942275,0.645704,0.403604,0.536013,0.536378,0.613167
1,-0.081586,-0.094962,-0.090605,-0.080890,-0.094307,-0.089937,-0.091860,-0.100504,-0.114275,-0.114040,...,0.043221,0.052041,0.033554,0.039472,0.014051,-0.057593,-0.092059,0.043800,0.035027,0.025233
2,-0.154685,-0.152784,-0.148737,-0.154215,-0.152308,-0.148249,-0.186795,-0.193686,-0.187003,-0.122805,...,-0.165588,-0.291924,-0.290712,-0.288318,-0.255837,-0.267913,-0.255946,-0.287121,-0.284955,-0.280676
3,-0.156643,-0.156666,-0.156522,-0.156179,-0.156202,-0.156058,-0.204861,-0.204790,-0.204718,-0.125660,...,-0.165588,-0.242793,-0.262878,-0.273303,-0.273969,-0.273969,-0.273969,-0.268832,-0.294398,-0.294447
4,-0.138847,-0.129233,-0.141798,-0.138328,-0.128684,-0.141289,-0.182486,-0.162965,-0.170262,-0.125630,...,-0.078297,-0.124641,-0.121939,-0.128903,-0.103807,-0.134192,-0.166740,-0.130025,-0.134049,-0.142831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.067129,-0.004851,-0.039281,-0.066388,-0.003917,-0.038454,-0.108082,-0.053838,-0.053694,-0.031401,...,-0.078382,-0.295238,-0.290994,-0.283410,-0.269524,-0.248313,-0.214854,-0.293044,-0.285459,-0.271905
199996,0.543173,0.413435,0.293117,0.545804,0.415664,0.294974,0.382148,0.281704,0.189316,1.484547,...,-0.153030,-0.125855,-0.126250,-0.126344,-0.221403,-0.222278,-0.225526,-0.136274,-0.178142,-0.194693
199997,-0.082159,-0.111169,0.032941,-0.081465,-0.110564,0.033992,-0.028263,-0.041417,0.140726,-0.120011,...,0.680322,1.732060,1.662368,1.694414,0.661477,0.127654,0.751383,1.658004,1.251180,1.126610
199998,-0.156775,-0.156775,-0.156776,-0.156311,-0.156311,-0.156312,-0.204960,-0.204960,-0.204960,-0.125987,...,-0.165588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
y_val.info()

<class 'pandas.core.series.Series'>
Index: 40000 entries, 119737 to 6584
Series name: end_cluster_3month
Non-Null Count  Dtype 
--------------  ----- 
40000 non-null  object
dtypes: object(1)
memory usage: 625.0+ KB


In [70]:
i=0
for train_idx, val_idx in cv.split(X, y):
    x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    cat_model.fit(
        x_train, y_train,
        eval_set=(x_val, y_val),
        cat_features=cat_features,
        use_best_model=True,
        verbose=True
     )

    y_pred_proba = cat_model.predict_proba(x_val)
    cat_model.save_model(f'my_meth/model_{i}.cbm')
    i+=1
    print('y_val')
    print(y_val)
    print('y_pred_proba')
    print(y_pred_proba)
    print('cat_model.classes_')
    print(cat_model.classes_)
    print('weights_dict')
    print(weights_dict)
    auc_score = weighted_roc_auc(y_val, y_pred_proba, cat_model.classes_, weights_dict)
    auc_scores.append(auc_score)

mean_auc = np.mean(auc_scores)
std_auc = np.std(auc_scores)
print(f"Средний взвешенный ROC AUC: {mean_auc:.3f} ± {std_auc:.3f}")


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8600836	best: 0.8600836 (0)	total: 627ms	remaining: 10m 25s
1:	total: 1.31s	remaining: 10m 53s
2:	total: 2.03s	remaining: 11m 14s
3:	total: 2.67s	remaining: 11m 3s
4:	total: 3.33s	remaining: 11m 2s
5:	test: 0.8933983	best: 0.8949130 (4)	total: 4.13s	remaining: 11m 23s
6:	total: 4.93s	remaining: 11m 39s
7:	total: 5.61s	remaining: 11m 35s
8:	total: 7.03s	remaining: 12m 53s
9:	total: 7.85s	remaining: 12m 57s
10:	test: 0.8987616	best: 0.8987616 (10)	total: 8.53s	remaining: 12m 47s
11:	total: 9.3s	remaining: 12m 45s
12:	total: 10.1s	remaining: 12m 49s
13:	total: 10.9s	remaining: 12m 46s
14:	total: 11.7s	remaining: 12m 49s
15:	test: 0.8991225	best: 0.9018365 (12)	total: 12.4s	remaining: 12m 43s
16:	total: 13.1s	remaining: 12m 34s
17:	total: 13.8s	remaining: 12m 34s
18:	total: 14.5s	remaining: 12m 31s
19:	total: 15.2s	remaining: 12m 24s
20:	test: 0.8992291	best: 0.9018365 (12)	total: 15.9s	remaining: 12m 20s
21:	total: 16.5s	remaining: 12m 13s
22:	total: 17.2s	remaining: 12m 8s
23:

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8507954	best: 0.8507954 (0)	total: 615ms	remaining: 10m 13s
1:	total: 1.28s	remaining: 10m 39s
2:	total: 1.94s	remaining: 10m 43s
3:	total: 2.58s	remaining: 10m 42s
4:	total: 3.42s	remaining: 11m 21s
5:	test: 0.8694784	best: 0.8694784 (5)	total: 4.1s	remaining: 11m 18s
6:	total: 4.8s	remaining: 11m 20s
7:	total: 5.55s	remaining: 11m 28s
8:	total: 6.26s	remaining: 11m 29s
9:	total: 6.97s	remaining: 11m 30s
10:	test: 0.8880822	best: 0.8880822 (10)	total: 7.79s	remaining: 11m 40s
11:	total: 8.57s	remaining: 11m 45s
12:	total: 9.23s	remaining: 11m 41s
13:	total: 9.86s	remaining: 11m 34s
14:	total: 10.5s	remaining: 11m 32s
15:	test: 0.9004100	best: 0.9004100 (15)	total: 11.3s	remaining: 11m 32s
16:	total: 12s	remaining: 11m 33s
17:	total: 12.7s	remaining: 11m 31s
18:	total: 13.6s	remaining: 11m 40s
19:	total: 14.4s	remaining: 11m 43s
20:	test: 0.9009223	best: 0.9021844 (18)	total: 15s	remaining: 11m 40s
21:	total: 15.7s	remaining: 11m 36s
22:	total: 16.4s	remaining: 11m 37s
23:	t

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	test: 0.8631911	best: 0.8631911 (0)	total: 603ms	remaining: 10m 1s
1:	total: 1.26s	remaining: 10m 30s
2:	total: 1.88s	remaining: 10m 24s
3:	total: 2.53s	remaining: 10m 30s
4:	total: 3.3s	remaining: 10m 56s
5:	test: 0.8979183	best: 0.8979183 (5)	total: 4.03s	remaining: 11m 7s
6:	total: 4.72s	remaining: 11m 10s
7:	total: 5.51s	remaining: 11m 23s
8:	total: 6.22s	remaining: 11m 24s
9:	total: 6.96s	remaining: 11m 28s
10:	test: 0.9075809	best: 0.9089322 (9)	total: 7.77s	remaining: 11m 38s
11:	total: 8.52s	remaining: 11m 41s
12:	total: 9.32s	remaining: 11m 47s
13:	total: 10s	remaining: 11m 44s
14:	total: 10.8s	remaining: 11m 49s
15:	test: 0.9166873	best: 0.9166873 (15)	total: 11.5s	remaining: 11m 46s
16:	total: 12.3s	remaining: 11m 53s
17:	total: 13s	remaining: 11m 49s
18:	total: 13.7s	remaining: 11m 49s
19:	total: 14.4s	remaining: 11m 47s
20:	test: 0.9236536	best: 0.9236536 (20)	total: 15.1s	remaining: 11m 44s
21:	total: 15.8s	remaining: 11m 41s
22:	total: 16.4s	remaining: 11m 35s
23:	tot

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8580947	best: 0.8580947 (0)	total: 615ms	remaining: 10m 14s
1:	total: 1.28s	remaining: 10m 39s
2:	total: 1.93s	remaining: 10m 42s
3:	total: 2.56s	remaining: 10m 38s
4:	total: 3.25s	remaining: 10m 47s
5:	test: 0.8801905	best: 0.8801905 (5)	total: 3.94s	remaining: 10m 53s
6:	total: 4.7s	remaining: 11m 6s
7:	total: 5.38s	remaining: 11m 7s
8:	total: 6.2s	remaining: 11m 23s
9:	total: 7.01s	remaining: 11m 34s
10:	test: 0.8924244	best: 0.8924244 (10)	total: 7.73s	remaining: 11m 35s
11:	total: 8.43s	remaining: 11m 34s
12:	total: 9.22s	remaining: 11m 40s
13:	total: 10s	remaining: 11m 47s
14:	total: 10.8s	remaining: 11m 48s
15:	test: 0.9004404	best: 0.9004404 (15)	total: 11.5s	remaining: 11m 47s
16:	total: 12.3s	remaining: 11m 53s
17:	total: 13s	remaining: 11m 49s
18:	total: 13.7s	remaining: 11m 45s
19:	total: 14.3s	remaining: 11m 39s
20:	test: 0.9044810	best: 0.9044810 (20)	total: 15s	remaining: 11m 37s
21:	total: 15.6s	remaining: 11m 34s
22:	total: 16.4s	remaining: 11m 34s
23:	total

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8391546	best: 0.8391546 (0)	total: 605ms	remaining: 10m 4s
1:	total: 1.28s	remaining: 10m 39s
2:	total: 1.98s	remaining: 10m 57s
3:	total: 2.64s	remaining: 10m 56s
4:	total: 3.33s	remaining: 11m 2s
5:	test: 0.8699942	best: 0.8699942 (5)	total: 4.09s	remaining: 11m 18s
6:	total: 4.72s	remaining: 11m 9s
7:	total: 5.46s	remaining: 11m 16s
8:	total: 6.13s	remaining: 11m 14s
9:	total: 6.88s	remaining: 11m 21s
10:	test: 0.8833645	best: 0.8833645 (10)	total: 7.57s	remaining: 11m 20s
11:	total: 8.35s	remaining: 11m 27s
12:	total: 9.15s	remaining: 11m 35s
13:	total: 9.86s	remaining: 11m 34s
14:	total: 10.5s	remaining: 11m 29s
15:	test: 0.8950946	best: 0.8962003 (14)	total: 11.2s	remaining: 11m 31s
16:	total: 12s	remaining: 11m 34s
17:	total: 12.6s	remaining: 11m 29s
18:	total: 13.2s	remaining: 11m 24s
19:	total: 13.9s	remaining: 11m 21s
20:	test: 0.9045070	best: 0.9045070 (20)	total: 14.5s	remaining: 11m 16s
21:	total: 15.1s	remaining: 11m 12s
22:	total: 15.9s	remaining: 11m 15s
23:	